In [ ]:
from matplotlib import pyplot as plt
from matplotlib.patches import Patch
from matplotlib.colors import to_rgb
from matplotlib.ticker import EngFormatter, FuncFormatter
from datetime import timedelta
import json

def to_time(t, _):
    m = int(t / 60)
    s = int(t) % 60
    return f"{m:02d}:{s:02d}"

with open("wtf6.json") as file:
    data = json.load(file)

In [ ]:
sorted_trivias = [t for t, v in sorted(data["last_values"].items(), key=lambda o: -o[1])]
highscore = sorted_trivias[:10]

for index, trivia in enumerate(highscore, start=1):
    plot = data["plot"][trivia]
    label = f"{index:}. {trivia}"
    x = [p[0] for p in plot]
    y = [p[1] * 100 for p in plot]
    plt.plot(x, y, label=label)

ax = plt.gca()
fig = plt.gcf()
fig.set_size_inches(15, 5)
ax.legend()
ax.grid(alpha=0.50, which="major")
ax.grid(alpha=0.25, which="minor")
ax.yaxis.set_major_formatter(EngFormatter())
ax.xaxis.set_major_formatter(FuncFormatter(to_time))

In [ ]:
import numpy as np
from matplotlib.colors import LinearSegmentedColormap
from matplotlib.transforms import offset_copy
from scipy.ndimage import gaussian_filter
from cycler import cycler
import matplotlib as mpl
mpl.rcParams['hatch.linewidth'] = 2.0  # previous svg hatch linewidth

cycle = cycler(color=['tab:blue', 'tab:red', 'tab:green', 'gold', 'deepskyblue', 'violet'])
legend = []
TOP = 6

old_heatmaps = [np.array(data["heatmap"][trivia], dtype=float) for trivia in highscore[:TOP]]
total_heatmap = np.ones(old_heatmaps[0].shape, dtype=float) * 1e-6

for heatmap in old_heatmaps:
    total_heatmap += heatmap
    
heatmaps = []
for heatmap in old_heatmaps:
    heatmap /= total_heatmap
    heatmap = np.kron(heatmap, np.ones((3, 3), dtype=float))
    heatmap = gaussian_filter(heatmap, sigma=2)
    heatmaps.append(np.flipud(heatmap))
    
for params, (index, trivia) in zip(cycle, enumerate(highscore, start=1)):
    legend.append(Patch(facecolor=params["color"], label=f"{index}. {trivia}"))
    
LEVELS = [
    (0.1, 0.2, "."),
    (0.2, 0.4, ".."),
    (0.4, 1.0, "o"),
    #(0.0, 0.0, "/"),
]

fig, ax = plt.subplots()
fig.set_size_inches(total_heatmap.shape[1] / 2.0, total_heatmap.shape[0] / 2.0)

colorz = (0, 0, 0 ,0)
for a, b, h in LEVELS:
    for params, heatmap, trivia in zip(cycle, heatmaps, highscore):
        color = to_rgb(params["color"]) + (1,)
        mask = (heatmap > a) & (heatmap <= b)
        ma = np.ma.masked_array(heatmap, ~mask)
        cmap = LinearSegmentedColormap.from_list("", [colorz, colorz])
        plt.pcolor(ma, linewidth=0, color=color, cmap=cmap, hatch=h)

#for a, b, h in LEVELS:
#    img = np.zeros((*heatmaps[0].shape, 4), dtype=float)
#    for params, heatmap, trivia in zip(cycle, heatmaps, highscore):
#        color = to_rgb(params["color"]) + (b,)
#        mask = (heatmap > a) & (heatmap <= b)
#        img[mask] = color
#    plt.imshow(img)

ax.set_facecolor("#888")
ax.set_xticks([])
ax.set_yticks([])
ax.legend(
    handles=legend,
    bbox_to_anchor=(0.5, 0.0),
    loc='upper center',
    ncol=3,
)